### Es 1
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv('data.csv')
print(df.shape)
df.head(10)

In [ ]:
df.dropna(subset=['passengers'], inplace=True)

In [ ]:
# Assegno un indice progressivo ai mesi
df["mese_numerico"] = range(len(df))

X = df[["mese_numerico"]].values
y = df["passengers"].values

plt.figure(figsize=(12, 6))
plt.plot(X, y, marker='o', linestyle='-')
plt.title('Passeggri')
plt.xlabel('Date')
plt.ylabel('Passeggeri')
plt.grid(True)
plt.show()

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=90, shuffle=False)

model_poly2 = make_pipeline(PolynomialFeatures(2), LinearRegression())
model_poly2.fit(X_train, y_train)
y_pred_poly2 = model_poly2.predict(X_test)

plt.figure(figsize=(10, 5))
plt.scatter(X, y, label="Dati", color="blue")
plt.plot(X_test, y_pred_poly2, label=f"Polinomiale grado 2", color="green")
plt.xlabel("Indice Mese")
plt.ylabel("Passeggeri")
plt.title("Regressione polinomiale")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error

mae =  mean_absolute_error(y_test, y_pred_poly2)
rmse =  root_mean_squared_error(y_test, y_pred_poly2)
print(f'MAE: {mae}\nRMSE: {rmse}')

### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import dash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go

# 1
x = np.linspace(-3, 3, 100)

# 2
noise = np.random.normal(0, 3, size=x.shape)
y = x**3 - x + noise

# costruisco un DataFrame
df = pd.DataFrame({'x': x, 'y': y})


def regressione(df, deg):
    X = df[['x']].values
    y = df['y'].values

    poly = PolynomialFeatures(degree=deg)
    X_poly = poly.fit_transform(X)
    model = LinearRegression().fit(X_poly, y)

    x_plot = np.linspace(df['x'].min(), df['x'].max(), 300)
    X_plot_poly = poly.transform(x_plot.reshape(-1, 1))
    y_plot = model.predict(X_plot_poly)
    return x_plot, y_plot

# 3
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Regressione Polinomiale Interattiva"),
    dcc.Slider(
        id='degree-slider',
        min=1,
        max=10,
        step=1,
        value=3,
        marks={i: str(i) for i in range(1, 11)},
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    dcc.Graph(id='poly-regression-plot')
])


@app.callback(
    Output('poly-regression-plot', 'figure'),
    Input('degree-slider', 'value')
)
def update_poly_graph(degree):
    x_plot, y_plot = regressione(df, degree)

    # Grafico Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['x'], y=df['y'],
        mode='markers',
        name='Dati osservati',
        marker=dict(color='black', opacity=0.6)
    ))
    fig.add_trace(go.Scatter(
        x=x_plot, y=y_plot,
        mode='lines',
        name=f'Regressione polinomiale (grado {degree})',
        line=dict(color='red', width=2)
    ))
    fig.update_layout(
        xaxis_title='x',
        yaxis_title='y',
        title=f"Adattamento polinomiale di grado {degree}"
    )
    return fig


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)